In [1]:
##install.packages("remotes")
remotes::install_github("traitecoevo/APCalign")


withr   (3.0.0 -> 3.0.1 ) [CRAN]
crayon  (1.5.2 -> 1.5.3 ) [CRAN]
openssl (2.1.1 -> 2.2.0 ) [CRAN]
curl    (5.2.0 -> 5.2.1 ) [CRAN]
stringi (1.8.3 -> 1.8.4 ) [CRAN]
arrow   (NA    -> 16.1.0) [CRAN]
readr   (2.1.4 -> 2.1.5 ) [CRAN]


Installing 7 packages: withr, crayon, openssl, curl, stringi, arrow, readr




The downloaded binary packages are in
	/var/folders/0l/lyqlsmz14tq61kh1j0_tqtg80000gq/T//RtmpVSx28c/downloaded_packages
── R CMD build ─────────────────────────────────────────────────────────────────────────────────────────
✔  checking for file ‘/private/var/folders/0l/lyqlsmz14tq61kh1j0_tqtg80000gq/T/RtmpVSx28c/remotes142b32ff88f66/traitecoevo-APCalign-dfcafde/DESCRIPTION’
─  preparing ‘APCalign’:
✔  checking DESCRIPTION meta-information
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
─  building ‘APCalign_1.0.2.tar.gz’
   


In [2]:
library(APCalign)

Download the stable version of the taxonomic resources

In [3]:
stable_resources <- load_taxonomic_resources(stable_or_current_data = "stable")


Downloading...

File downloaded successfully.

File downloaded successfully.



Loading resources into memory...



...done

